In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cp -r /content/drive/MyDrive/celebdf /content/

In [3]:
import os

data_path = '/content/data'
for folder in ['train', 'test', 'val']:
  os.makedirs(os.path.join(data_path, folder), exist_ok = True)

In [4]:
from tqdm import tqdm

source_path = "/content/celebdf"
for mode in os.listdir(source_path):
  dest_real = os.path.join(data_path,mode, "real")
  dest_fake = os.path.join(data_path,mode, "fake")
  # make mode dirs
  os.makedirs(dest_real, exist_ok = True)
  os.makedirs(dest_fake, exist_ok = True)

  dest_mode = os.path.join(source_path, mode)
  for dest_zip in tqdm(os.listdir(dest_mode), desc=f"{str(mode)}", leave=False):
    dest_zip_file_path = os.path.join(dest_mode,dest_zip)
    if "real" in dest_zip:
      !unzip -q -j {dest_zip_file_path} -d {dest_real}
    elif "fake" in dest_zip:
      !unzip -q -j {dest_zip_file_path} -d {dest_fake}

In [5]:
!cp /content/drive/MyDrive/leonardo_preprocessed/preprocessed_leonardo.zip .

In [6]:
!unzip /content/preprocessed_leonardo.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/preprocessed_leonardo/train/9239690f-29b9-4348-a1fd-e51821a79f69__a5678c0c.png  
  inflating: content/preprocessed_leonardo/train/139c7d57-cc0a-47de-a97a-876a87ca0f27__fbe9e176.png  
  inflating: content/preprocessed_leonardo/train/01a11547-31d2-4fd1-9d41-4c85f77ee6f5__c00db466.png  
  inflating: content/preprocessed_leonardo/train/c25aa048-7ac7-461c-9f2b-eb69c151eeec__3bd09d5a.png  
  inflating: content/preprocessed_leonardo/train/199655e0-fae0-4da1-8832-89086a06cf36__0ec061b2.png  
  inflating: content/preprocessed_leonardo/train/85172d1b-d407-48fb-b48d-1e01a634c756__6e7adb36.png  
  inflating: content/preprocessed_leonardo/train/b6b1b451-236c-4779-9da6-ebd595656a0a__075fe8dc.png  
  inflating: content/preprocessed_leonardo/train/e02a89eb-c85d-4d01-96f2-c7804b101e4b__aff7679a.png  
  inflating: content/preprocessed_leonardo/train/30cdef4f-7b0a-4fbf-904a-0d735f445636__59c2ed74.png  
  inflating: content/preprocess

In [7]:
from glob import glob
import shutil

for file_p in sorted(glob("/content/content/preprocessed_leonardo/train/*")):
  src = file_p
  dst = file_p.replace("/content/content/preprocessed_leonardo/train/", "/content/data/train/fake/")
  shutil.move(src, dst)
  # break

In [8]:
from glob import glob
import shutil

for file_p in sorted(glob("/content/content/preprocessed_leonardo/val/*")):
  src = file_p
  dst = file_p.replace("/content/content/preprocessed_leonardo/val/", "/content/data/val/fake/")
  shutil.move(src, dst)

In [9]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm

In [11]:
# Parameters
data_dir = '/content/data'  # Replace with your dataset path
checkpoint_dir = '/content/drive/MyDrive/convnext_ckpts/trained_on_celebdf_n_leonardo'
os.makedirs(checkpoint_dir, exist_ok=True)
batch_size = 32
num_epochs = 10
learning_rate = 1e-4
checkpoint_interval = 0.1  # Save checkpoint after every 10% of data

In [12]:
# Dataset and DataLoader
input_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [13]:
train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=input_transforms)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

val_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'val'), transform=input_transforms)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [14]:
model = timm.create_model('convnext_base', pretrained=True, num_classes=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

In [15]:
for name, param in model.named_parameters():
    param.requires_grad = False
    if name.startswith('head') or "stages.3.blocks.2" in name:
        param.requires_grad = True

# model.load_state_dict(torch.load("/content/drive/MyDrive/convnext_ckpts/trained_only_on_celebdf/epoch_1_step_25000.pth"))
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=512, out_features=128, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), g

In [16]:
def train():
    model.train()
    total_steps = len(train_loader)
    checkpoint_steps = 10000

    for epoch in range(num_epochs):
        running_loss = 0.0
        for step, (inputs, labels) in enumerate(train_loader):
            print(f"Epoch: {epoch} Step {step} / {total_steps}")
            inputs, labels = inputs.to(device), labels.float().to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if (step + 1) % checkpoint_steps == 0:
                checkpoint_path = os.path.join(checkpoint_dir, f'epoch_{epoch+1}_step_{step+1}.pth')
                torch.save(model.state_dict(), checkpoint_path)
                print(f'Checkpoint saved at {checkpoint_path}')

        avg_train_loss = running_loss / total_steps
        with open("/content/drive/MyDrive/convnext_ckpts/trained_on_celebdf_n_leonardo/train_report.txt", "a+") as h:
          h.writelines(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f} ')
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}')

        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_running_loss = 0.0
        with torch.no_grad():  # No need to track gradients during validation
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.float().to(device)

                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels)
                val_running_loss += loss.item()

        avg_val_loss = val_running_loss / len(val_loader)
        with open("/content/drive/MyDrive/convnext_ckpts/trained_on_celebdf_n_leonardo/train_report.txt", "a+") as h:
          h.writelines(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f} \n')
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

        # Switch back to training mode
        model.train()

In [17]:
train()

Streaming output truncated to the last 5000 lines.
Epoch: 9 Step 21206 / 26204
Epoch: 9 Step 21207 / 26204
Epoch: 9 Step 21208 / 26204
Epoch: 9 Step 21209 / 26204
Epoch: 9 Step 21210 / 26204
Epoch: 9 Step 21211 / 26204
Epoch: 9 Step 21212 / 26204
Epoch: 9 Step 21213 / 26204
Epoch: 9 Step 21214 / 26204
Epoch: 9 Step 21215 / 26204
Epoch: 9 Step 21216 / 26204
Epoch: 9 Step 21217 / 26204
Epoch: 9 Step 21218 / 26204
Epoch: 9 Step 21219 / 26204
Epoch: 9 Step 21220 / 26204
Epoch: 9 Step 21221 / 26204
Epoch: 9 Step 21222 / 26204
Epoch: 9 Step 21223 / 26204
Epoch: 9 Step 21224 / 26204
Epoch: 9 Step 21225 / 26204
Epoch: 9 Step 21226 / 26204
Epoch: 9 Step 21227 / 26204
Epoch: 9 Step 21228 / 26204
Epoch: 9 Step 21229 / 26204
Epoch: 9 Step 21230 / 26204
Epoch: 9 Step 21231 / 26204
Epoch: 9 Step 21232 / 26204
Epoch: 9 Step 21233 / 26204
Epoch: 9 Step 21234 / 26204
Epoch: 9 Step 21235 / 26204
Epoch: 9 Step 21236 / 26204
Epoch: 9 Step 21237 / 26204
Epoch: 9 Step 21238 / 26204
Epoch: 9 Step 21239 / 262